# gold_to_postgres.ipynb
Load Gold Layer Parquet → Write into PostgreSQL

In [1]:
import os
import import_ipynb
import pandas as pd
from sqlalchemy import create_engine
from config import *

GOLD_LAYER_PATH = os.path.join(GOLD_LAYER,"cleaned_merged_gold.parquet")
df_gold = pd.read_parquet(GOLD_LAYER_PATH, engine="pyarrow")
print("[INFO] Loaded DataFrame shape:", df_gold.shape)

engine = create_engine(f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}")

df_gold.to_sql(TABLE_NAME, engine, if_exists='replace', index=False)
print(f"[INFO] Successfully written to Postgres table: {TABLE_NAME}")

[INFO] Loaded DataFrame shape: (681493, 10)
[INFO] Successfully written to Postgres table: agro_kpi


In [2]:
import requests
import pandas as pd
import time

def get_location_info(lat, lon):
    """
    Reverse geocode latitude & longitude into country and city_district.
    Uses OpenStreetMap Nominatim API (free, rate-limited).
    """
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lat,
        "lon": lon,
        "format": "json",
        "zoom": 10,   # control detail level
        "addressdetails": 1
    }
    headers = {"User-Agent": "geo-app"}
    
    try:
        response = requests.get(url, params=params, headers=headers, timeout=5)
        if response.status_code == 200:
            data = response.json()
            address = data.get("address", {})
            return address.get("country"), address.get("city_district") or address.get("city")
        else:
            print(f"[WARN] Failed for lat={lat}, lon={lon}, status={response.status_code}")
            return None, None
    except Exception as e:
        print(f"[ERROR] {e} for lat={lat}, lon={lon}")
        return None, None

# --- Example DataFrame ---

GOLD_LAYER_PATH = os.path.join(GOLD_LAYER,"cleaned_merged_gold.parquet")
df_gold = pd.read_parquet(GOLD_LAYER_PATH, engine="pyarrow")


# --- Enrich DataFrame ---
countries = []
cities = []

for idx, row in df_gold.head().iterrows():
    country, city = get_location_info(row["lat"], row["lon"])
    countries.append(country)
    cities.append(city)
    time.sleep(1)

# Apply to the test dataframe
df_test = df_gold.head().copy()
df_test["country"] = countries
df_test["city_district"] = cities

print(df_test)


         lat         lon  year  month        sst         poc       pic  \
0 -25.979176  153.187500  2025      8  21.609999   71.599907  0.000014   
1 -25.979176  153.229172  2025      8  21.775000   80.199905  0.000182   
2 -25.979176  153.270844  2025      8  22.434999   81.799904  0.000194   
3 -25.979176  153.312500  2025      8  22.619999  121.799904  0.001074   
4 -25.979176  153.354172  2025      8  22.619999  138.599899  0.001340   

   aot_862   chlor_a  Kd_490    country city_district  
0   0.0391  0.258860  0.0452  Australia          None  
1   0.0374  0.282766  0.0492  Australia          None  
2   0.0342  0.290534  0.0516  Australia          None  
3   0.0289  0.449613  0.0672  Australia          None  
4   0.0282  0.536851  0.0748  Australia          None  
